In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(pROC)
}))


In [2]:
run_name <- '1_p_factor_v1'
phenotype <- 'HC382'

data_d <- file.path(
    '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test', 
    run_name, phenotype, '1_fit_w_val'
)
refit <- F

# output
p_val_f <- sprintf('AUC_diff_p.%s.tsv', phenotype)
eval_f <- sprintf('AUC_diff_p_eval.%s.tsv', phenotype)


In [3]:
# input and parameters

#data_d <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test'
phe_f <- '/scratch/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.20200828.phe.zst'
covariates       <- c('age', 'sex', paste0('PC', 1:10))
refit_split_strs <- c('non_british_white', 'african', 's_asian', 'e_asian')
sscore_f             <- file.path(data_d, '__PHENOTYPE__.sscore.zst')
snpnet_BETAs_f       <- file.path(data_d, 'snpnet.tsv')
snpnet_covar_BETAs_f <- file.path(data_d, 'snpnet.covars.tsv')
family <- ifelse((startsWith(phenotype, 'INI') | startsWith(phenotype, 'QT_FC')), 'gaussian', 'binomial')

# output
eval_f <- file.path(data_d, 'snpnet.eval.tsv')
plot_f <- file.path(data_d, 'snpnet.plot.pdf')
percentile_f <- file.path(data_d, 'snpnet.percentile.tsv')


In [4]:
source('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/snpnet/helpers/snpnet_misc.R')

# read the raw phenotype file
fread(
    cmd=paste(cat_or_zcat(phe_f), phe_f,  '|', 'sed -e "s/^#//g"'),
    select=c('FID', 'IID', 'split', covariates, phenotype),
    colClasses = c('FID'='character', 'IID'='character'),
    data.table=F
) -> phe_df

if(refit){
    phe_df %>%
    mutate(
        split = if_else(split %in% c('train', 'val'), 'train_val', split)
    ) -> phe_df
}

# read PRS and covariate-based score
phe_df %>%
compute_phe_score_df(
    phenotype,
    str_replace_all(sscore_f, '__PHENOTYPE__', phenotype),
    str_replace_all(snpnet_covar_BETAs_f, '__PHENOTYPE__', phenotype),
    covariates, family, refit_split_strs
) -> phe_score_df


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [5]:
Sys.glob(file.path(data_d, 'results', '*.sscore.zst'), dirmark = FALSE) %>%
lapply(function(x){as.integer(str_replace_all(basename(x), '^snpnet.lambda|.sscore.zst$', ''))}) %>%
simplify() %>% sort() -> lambda_idxs 


In [6]:
lambda_idxs 

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60

In [7]:
lambda_idxs %>%
lapply(function(lambda_idx){
    file.path(data_d, 'results', sprintf('snpnet.lambda%d.sscore.zst', lambda_idx)) %>%
    read_PRS() %>%
    rename(!!sprintf('geno_%d', lambda_idx) := 'geno_score')
}) %>%
reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))}) -> geno_score_intermediates_df


In [8]:
phe_score_df %>% 
left_join(geno_score_intermediates_df, by=c('FID', 'IID')) -> phe_score_intermediate_df


In [9]:
phe_score_intermediate_df %>%
filter(split == 'val') %>%
mutate(geno_covar_score = geno_score + covar_score) -> sdf


In [10]:
roc_tests <- list()


In [11]:
roc.test(
    roc(sdf$phe, sdf[['covar_score']], levels=c('control'=1, 'case'=2), direction='<'),
    roc(sdf$phe, sdf[['geno_covar_score']], levels=c('control'=1, 'case'=2), direction='<')
) -> roc_tests[['geno_covar_vs_covar']]


In [12]:
roc.test(
    roc(sdf$phe, sdf[['covar_score']], levels=c('control'=1, 'case'=2), direction='<'),
    roc(sdf$phe, sdf[['geno_1']], levels=c('control'=1, 'case'=2), direction='<')
) -> roc_tests[['geno_1_vs_covar']]


In [13]:
for(lambda_idx in lambda_idxs[lambda_idxs != 1]){
    roc.test(
        roc(sdf$phe, sdf[[sprintf('geno_%d', lambda_idx - 1)]], levels=c('control'=1, 'case'=2), direction='<'),
        roc(sdf$phe, sdf[[sprintf('geno_%d', lambda_idx    )]], levels=c('control'=1, 'case'=2), direction='<')
    ) -> roc_tests[[sprintf('geno_%d_vs_geno_%d', lambda_idx, lambda_idx - 1)]]
}


In [14]:
roc_p_vals <- data.frame(
    name = names(roc_tests),
    p = names(roc_tests) %>%
    lapply(function(k){
        (roc_tests[[k]])$p.value
    }) %>% simplify(),
    stringsAsFactors=F
) %>%
separate(name, c('score2', 'score1'), sep='_vs_', remove=F) %>%
select(name, score1, score2, p)


In [15]:
roc_p_vals %>%
rename('#name' = 'name') %>%
fwrite(p_val_f, sep='\t', na = "NA", quote=F)


In [16]:
roc_tests[['geno_covar_vs_covar']]


	DeLong's test for two correlated ROC curves

data:  roc(sdf$phe, sdf[["covar_score"]], levels = c(control = 1, case = 2),  and roc(sdf$phe, sdf[["geno_covar_score"]], levels = c(control = 1,     direction = "<") and     case = 2), direction = "<")
Z = -16.507, p-value < 2.2e-16
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.5281458   0.6189194 


In [17]:
roc_p_vals %>%
tail()

,name,score1,score2,p
,<chr>,<chr>,<chr>,<dbl>
56,geno_55_vs_geno_54,geno_54,geno_55,0.01316363
57,geno_56_vs_geno_55,geno_55,geno_56,0.10500133
58,geno_57_vs_geno_56,geno_56,geno_57,0.17819255
59,geno_58_vs_geno_57,geno_57,geno_58,0.35657788
60,geno_59_vs_geno_58,geno_58,geno_59,0.76033081
61,geno_60_vs_geno_59,geno_59,geno_60,0.36920904


- We currently have `lambda idx = 58` as the best model
- `lambda idx = 55` may be the optimal

In [18]:
# evaluate the predictive performance
55:60 %>% lapply(function(la_idx){
    phe_score_intermediate_df %>%
    select(-geno_score) %>%
    rename('geno_score'=sprintf('geno_%d', la_idx)) %>%
    eval_performance(
        phenotype,
        file.path(data_d, 'results', sprintf('snpnet.lambda%d.tsv', la_idx)),
        family
    ) %>%
    mutate(
        lambda_idx = la_idx
    ) %>%
    select(phenotype_name, split, lambda_idx, geno, covar, geno_covar, geno_delta, n_variables, case_n, control_n)
}) %>% bind_rows() -> eval_intermediate_df


In [19]:
eval_intermediate_df %>%
rename('#phenotype_name' = 'phenotype_name') %>%
fwrite(eval_f, sep='\t', na = "NA", quote=F)


In [20]:
eval_intermediate_df %>%
filter(split %in% c('train', 'test', 'val')) %>%
left_join(data.frame(split=c('train', 'val', 'test'), order=1:3, stringsAsFactors=F), by='split') %>%
arrange(lambda_idx, order) %>% select(-order)

phenotype_name,split,lambda_idx,geno,covar,geno_covar,geno_delta,n_variables,case_n,control_n
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
HC382,train,55,0.7323389,0.5387748,0.7312488,0.19247405,4764,31741,204250
HC382,val,55,0.6139289,0.5281458,0.6172363,0.08909049,4764,4553,29160
HC382,test,55,0.6120284,0.5352619,0.6163027,0.08104073,4764,8934,58491
HC382,train,56,0.7462311,0.5387748,0.7451270,0.20635225,5631,31741,204250
HC382,val,56,0.6145204,0.5281458,0.6179748,0.08982901,5631,4553,29160
HC382,test,56,0.6122403,0.5352619,0.6166296,0.08136768,5631,8934,58491
HC382,train,57,0.7598510,0.5387748,0.7588080,0.22003326,6564,31741,204250
HC382,val,57,0.6149959,0.5281458,0.6185883,0.09044256,6564,4553,29160
HC382,test,57,0.6122056,0.5352619,0.6166757,0.08141378,6564,8934,58491


In [21]:
roc_tests[['geno_56_vs_geno_55']]


	DeLong's test for two correlated ROC curves

data:  roc(sdf$phe, sdf[[sprintf("geno_%d", lambda_idx - 1)]], levels = c(control = 1,  and roc(sdf$phe, sdf[[sprintf("geno_%d", lambda_idx)]], levels = c(control = 1,     case = 2), direction = "<") and     case = 2), direction = "<")
Z = -1.6211, p-value = 0.105
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.6139289   0.6145204 
